In [4]:
import sys
import gi
import logging

gi.require_version("GLib", "2.0")
gi.require_version("GObject", "2.0")
gi.require_version("Gst", "1.0")

from gi.repository import Gst, GLib, GObject

uri_name='rtsp://127.0.0.1:8554/test'
file='/home/fkurt/Murat/ElephantsDream.mp4'

def cb_newpad(demux, src, sink):
    print("In cb_newpad\n")
    caps=src.get_current_caps()
    gststruct=caps.get_structure(0)
    gstname=gststruct.get_name()
    features=caps.get_features(0)

    print("gstname=",gstname)
    if(gstname.find("video")!=-1):
        print("features=",features)
        sink_pad=sink.get_static_pad("sink")
        if not src.link(sink_pad):
            sys.stderr.write("Failed to link decoder src pad to source bin ghost pad\n")


import sys
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst

# http://docs.gstreamer.com/display/GstSDK/Basic+tutorial+7%3A+Multithreading+and+Pad+Availability


def main():
    # initialize GStreamer
    Gst.init(sys.argv)

    # create the elements
    audio_source = Gst.ElementFactory.make("audiotestsrc", "audio_source")
    tee = Gst.ElementFactory.make("tee", "tee")
    audio_queue = Gst.ElementFactory.make("queue", "audio_queue")
    audio_convert = Gst.ElementFactory.make("audioconvert", "audio_convert")
    audio_resample = Gst.ElementFactory.make("audioresample", "audio_resample")
    audio_sink = Gst.ElementFactory.make("autoaudiosink", "audio_sink")
    video_queue = Gst.ElementFactory.make("queue", "video_queue")
    visual = Gst.ElementFactory.make("wavescope", "visual")
    video_convert = Gst.ElementFactory.make("videoconvert", "video_convert")
    video_sink = Gst.ElementFactory.make("autovideosink", "video_sink")

    # create the empty pipeline
    pipeline = Gst.Pipeline.new("test-pipeline")

    if (not pipeline or not audio_source or not tee or not audio_queue
            or not audio_convert or not audio_resample or not audio_sink
            or not video_queue or not visual or not video_convert
            or not video_sink):
        print("ERROR: Not all elements could be created.")
        sys.exit(1)

    # configure elements
    audio_source.set_property("freq", 215.0)
    visual.set_property("shader", 0)
    visual.set_property("style", 1)

    # link all elements that can be automatically linked because they have
    # always pads
    pipeline.add(audio_source, tee, audio_queue, audio_convert, audio_resample,
                 audio_sink, video_queue, visual, video_convert, video_sink)

    ret = audio_source.link(tee)
    ret = ret and audio_queue.link(audio_convert)
    ret = ret and audio_convert.link(audio_resample)
    ret = ret and audio_resample.link(audio_sink)
    ret = ret and video_queue.link(visual)
    ret = ret and visual.link(video_convert)
    ret = ret and video_convert.link(video_sink)

    if not ret:
        print("ERROR: Elements could not be linked")
        sys.exit(1)

    # manually link the tee, which has "Request" pads
    tee_src_pad_template = tee.get_pad_template("src_%u")
    tee_audio_pad = tee.request_pad(tee_src_pad_template, None, None)

    print(
        "Obtained request pad {0} for audio branch".format(
            tee_audio_pad.get_name()))

    audio_queue_pad = audio_queue.get_static_pad("sink")
    tee_video_pad = tee.request_pad(tee_src_pad_template, None, None)
    print(
        "Obtained request pad {0} for video branch".format(
            tee_video_pad.get_name()))
    video_queue_pad = video_queue.get_static_pad("sink")

    if (tee_audio_pad.link(audio_queue_pad) != Gst.PadLinkReturn.OK
            or tee_video_pad.link(video_queue_pad) != Gst.PadLinkReturn.OK):
        print("ERROR: Tee could not be linked")
        sys.exit(1)

    # one could use link() to link elements with"Reuest" pads automatically
    # instead of manually (see above), as it will internally
    # request the pads
    # tee.link(audio_queue)
    # tee.link(video_queue)

    # start playing
    pipeline.set_state(Gst.State.PLAYING)

    # wait until error or EOS
    terminate = False
    bus = pipeline.get_bus()
    while True:
        try:
            msg = bus.timed_pop_filtered(
                0.5 * Gst.SECOND,
                Gst.MessageType.ERROR | Gst.MessageType.EOS)
            if msg:
                terminate = True
        except KeyboardInterrupt:
            terminate = True

        if terminate:
            break

    pipeline.set_state(Gst.State.NULL)

if __name__ == '__main__':
    main()

Obtained request pad src_0 for audio branch
Obtained request pad src_1 for video branch


In [5]:
import logging
from threading import Thread, Event

from tools.application_init import application_init

application_init()

from gi.repository import Gst, GLib
from tools.runner import Runner

log = logging.getLogger("main")

log.info("building pipeline")
pipeline = Gst.Pipeline.new()
caps_audio = Gst.Caps.from_string("audio/x-raw,format=S16LE,rate=48000,channels=2")
caps_audio_be = Gst.Caps.from_string("audio/x-raw,format=S16BE,rate=48000,channels=2")
caps_rtp = Gst.Caps.from_string("application/x-rtp,clock-rate=48000,media=audio,encoding-name=L16,channels=2")

testsrc = Gst.ElementFactory.make("audiotestsrc", "testsrc1")
testsrc.set_property("is-live", True)
testsrc.set_property("freq", 220)
pipeline.add(testsrc)

tee = Gst.ElementFactory.make("tee")  # (1)
tee.set_property("allow-not-linked", True)
pipeline.add(tee)
testsrc.link_filtered(tee, caps_audio)

playback_internal = False  # (2)
if playback_internal:
    sink = Gst.ElementFactory.make("autoaudiosink")
    pipeline.add(sink)
    tee.link(sink)


# audioconvert ! {rawcaps_be} ! rtpL16depay ! udpsink port=…
def create_bin(port):
    log.info("Creating RTP-Bin for Port %d" % port)
    txbin = Gst.Bin.new("tx-bin-%d" % port)
    log.debug(txbin)

    log.info("Creating queue")
    queue = Gst.ElementFactory.make("queue")  # (3)
    log.debug(queue)

    log.info("Adding queue to bin")
    log.debug(txbin.add(queue))

    log.info("Creating audioconvert")
    audioconvert = Gst.ElementFactory.make("audioconvert")
    log.debug(audioconvert)

    log.info("Adding audioconvert to bin")
    log.debug(txbin.add(audioconvert))

    log.info("Linking queue to audioconvert")
    log.debug(queue.link(audioconvert))

    log.info("Creating payloader")
    payloader = Gst.ElementFactory.make("rtpL16pay")
    log.debug(payloader)

    log.info("Adding payloader to bin")
    log.debug(txbin.add(payloader))

    log.info("Linking audioconvert to payloader")
    log.debug(audioconvert.link_filtered(payloader, caps_audio_be))

    log.info("Creating udpsink")
    udpsink = Gst.ElementFactory.make("udpsink")
    log.debug(payloader)
    udpsink.set_property("host", "127.0.0.1")  # (4)
    udpsink.set_property("port", port)

    log.info("Adding udpsink to bin")
    log.debug(txbin.add(udpsink))

    log.info("Linking payloader to udpsink")
    log.debug(payloader.link(udpsink))

    log.info("Selecting Input-Pad")
    sink_pad = queue.get_static_pad("sink")
    log.debug(sink_pad)

    log.info("Creating Ghost-Pad")
    ghost_pad = Gst.GhostPad.new("sink", sink_pad)
    log.debug(ghost_pad)

    log.info("Adding Ghost-Pad to Bin")
    log.debug(txbin.add_pad(ghost_pad))

    return txbin


def add_bin(port):
    log.info("Adding RTP-Bin for Port %d to the Pipeline" % port)
    Gst.debug_bin_to_dot_file_with_ts(pipeline, Gst.DebugGraphDetails.ALL, "add_bin_%u_before" % port)

    log.info("Creating Bin")
    txbin = create_bin(port)
    log.info("Created Bin")
    log.debug(txbin)

    log.info("Adding bin to pipeline")
    log.debug(pipeline.add(txbin))

    log.info("Syncing Bin-State with Parent")
    log.debug(txbin.sync_state_with_parent())

    log.info("Linking bin to mixer")
    tee.link(txbin)

    Gst.debug_bin_to_dot_file_with_ts(pipeline, Gst.DebugGraphDetails.ALL, "add_bin_%u_after" % port)
    log.info("Added RTP-Bin for Port %d to the Pipeline" % port)


def remove_bin(port):
    log.info("Removing RTP-Bin for Port %d to the Pipeline" % port)
    Gst.debug_bin_to_dot_file_with_ts(pipeline, Gst.DebugGraphDetails.ALL, "remove_bin_%u_before" % port)

    log.info("Selecting Bin")
    txbin = pipeline.get_by_name("tx-bin-%d" % port)
    log.debug(txbin)

    log.info("Selecting Ghost-Pad")
    ghostpad = txbin.get_static_pad("sink")
    log.debug(ghostpad)

    log.info("Selecting Tee-Pad (Peer of Ghost-Pad)")
    teepad = ghostpad.get_peer()
    log.debug(teepad)

    def blocking_pad_probe(pad, info):
        log.info("Stopping Bin")
        log.debug(txbin.set_state(Gst.State.NULL))

        log.info("Removing Bin from Pipeline")
        log.debug(pipeline.remove(txbin))

        log.info("Releasing Tee-Pad")
        log.debug(tee.release_request_pad(teepad))

        Gst.debug_bin_to_dot_file_with_ts(pipeline, Gst.DebugGraphDetails.ALL, "remove_bin_%u_after" % port)
        log.info("Removed RTP-Bin for Port %d to the Pipeline" % port)

        return Gst.PadProbeReturn.REMOVE

    log.info("Configuring Blocking Probe on teepad")
    teepad.add_probe(Gst.PadProbeType.BLOCK, blocking_pad_probe)  # (5)


stop_event = Event()


def timed_sequence():
    log.info("Starting Sequence")

    num_ports = 3
    timeout = 0.2
    while True:
        for i in range(0, num_ports):
            if stop_event.wait(timeout): return
            log.info("Scheduling add_bin for Port %d", 15000 + i)
            GLib.idle_add(add_bin, 15000 + i)

        for i in range(0, num_ports):
            if stop_event.wait(timeout): return
            log.info("Scheduling remove_bin for Port %d", 15000 + i)
            GLib.idle_add(remove_bin, 15000 + i)


t = Thread(target=timed_sequence, name="Sequence")
t.start()

runner = Runner(pipeline)
runner.run_blocking()

stop_event.set()
t.join()

ModuleNotFoundError: No module named 'tools.application_init'; 'tools' is not a package